In [ ]:
# 1. 导入库（第二章知识点相关）
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# 2. 读取数据（参考第一张图片的文件路径）
df = pd.read_csv(r"D:/Download/all_anonymized_2015_11_2017_03.csv", encoding="utf-8")

# 3. 缺失值统计
missing_stats = pd.DataFrame({
    "缺失值数量": df.isna().sum(),
    "缺失率": (df.isna().sum() / len(df)).round(4) * 100
}).sort_values("缺失率", ascending=False)
print("缺失值统计：\n", missing_stats)

# 4. 数值型字段填补示例（mileage：分组中位数）
# 按「品牌-车型-生产年份」分组取中位数
df["mileage_filled"] = df.groupby(["maker", "model", "manufacture_year"])["mileage"].transform(
    lambda x: x.fillna(x.median())
)
# 剩余缺失用全局中位数填补
df["mileage_filled"] = df["mileage_filled"].fillna(df["mileage"].median())

# 5. 模型填补示例（price_eur：GBDT）
# 准备训练数据（无缺失的样本）
train_data = df.dropna(subset=["price_eur"]).copy()
X_train = train_data[["manufacture_year", "mileage_filled", "engine_displacement", "engine_power"]]
y_train = train_data["price_eur"]

# 训练GBDT模型
gbdt = GradientBoostingRegressor(random_state=42)
gbdt.fit(X_train, y_train)

# 预测缺失值
missing_mask = df["price_eur"].isna()
X_pred = df.loc[missing_mask, ["manufacture_year", "mileage_filled", "engine_displacement", "engine_power"]]
df.loc[missing_mask, "price_eur_filled"] = gbdt.predict(X_pred)
# 非缺失值直接保留
df.loc[~missing_mask, "price_eur_filled"] = df.loc[~missing_mask, "price_eur"]

# 6. 缺失值信息提取（missing_pattern）
# 选择缺失率>10%的字段
high_missing_cols = missing_stats[missing_stats["缺失率"] > 10].index.tolist()
# 构建缺失哑变量
missing_dummies = df[high_missing_cols].isna().astype(int).add_prefix("missing_")
# 拼接二进制模式
df["missing_pattern_bin"] = missing_dummies.astype(str).apply(lambda x: x.str.cat(), axis=1)
# 转换为十进制
df["missing_pattern"] = df["missing_pattern_bin"].apply(lambda x: int(x, 2))

# 7. 保存处理后的数据
df.to_csv("processed_used_cars.csv", index=False, encoding="utf-8")
print("处理后的数据已保存！")